In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [0]:
import pyspark.sql.functions as F

In [0]:
df = (spark
      .read
      .option('header','true')
      .option("nullValue",np.nan)
      .option("nanValue",np.nan)
      .csv('/FileStore/tables/Regression_final.csv')
      )

In [0]:
columns_tokeep = ['Identifier','Component']

In [0]:
#To filter the dataset with Status as 'M','R','V' as per business confirmation
df_filtered = (df
              .filter( (F.col('Status') == "M") |(F.col('Status') == "R") |(F.col('Status') == "V"))
              .filter( (F.col('Severity') == 1) |(F.col('Severity') == 2) |(F.col('Severity') == 3))
               )

In [0]:
print (df_filtered.cache().count(),len(df_filtered.columns))

(14201, 247)

In [0]:
#replcaed the null values of a target variable with N as per confirmation from business#

df_filtered = df_filtered.fillna('N',subset=['Regression'])

In [0]:
COUNT = df_filtered.cache().count()

In [0]:
#To Remove the columns with null values >50%

isNullStatCol = [F.sum(F.col(c).isNull().cast('int')).alias(c)  for c in df_filtered.columns]
isNullMeanStatCol = [(100*F.mean(F.col(c).isNull().cast('int'))).alias(c)  for c in df_filtered.columns]
na50cols = {k:v for k,v in df_filtered.select(*isNullMeanStatCol).head().asDict().items() if v > 50 and c not in columns_tokeep}
df_filtered3 = df_filtered.select([c for c in df_filtered.columns if c not in na50cols])

In [0]:
print (df_filtered3.cache().count(),len(df_filtered3.columns))

(14201, 105)

In [0]:
#To Remove the columns with the same value throughout the dataset

nuniqueCol = [F.countDistinct(F.col(c)).alias(c)  for c in df_filtered3.columns]
nunique = {k:v for k,v in df_filtered3.select(*nuniqueCol).head().asDict().items() if v == 1 and k not in columns_tokeep} 
df_filtered4 = df_filtered3.select([c for c in df_filtered3.columns if c not in nunique])

In [0]:
print (df_filtered4.cache().count(),len(df_filtered4.columns))

(14201, 88)

In [0]:
#To Remove the columns with the unique value >500 throughout the dataset
n500unique = {k:v for k,v in df_filtered3.select(*nuniqueCol).head().asDict().items() if (v >= 500) and (k not in columns_tokeep)} 
df_filtered5 = df_filtered4.select([c for c in df_filtered4.columns if c not in n500unique])


In [0]:
print (df_filtered5.cache().count(),len(df_filtered5.columns))

(14201, 57)

In [0]:
#To Remove the duplicate colums with same values
df_filtered6=df_filtered5.drop('DE-manager','DEPriorityDesc','DTPT-manager','Dev-escape-activity-display',
                   'Dev-escape-resolver-opinion','DE-manager-org','DTPT-manager-org','Found-during',
                   'Not-customer-visible-reason','Original-found-during','Severity-desc','org','Class')
print df_filtered6.cache().count(),len(df_filtered6.columns)

14201 45

In [0]:
#To Impute the continous variables with the median 
from pyspark.ml.feature import Imputer
df_filtered7 = (df_filtered6
                         .withColumn("Mean-time-to-resolved",F.col("Mean-time-to-resolved").cast('float'))
                          .withColumn("Mean-time-to-assigned",F.col("Mean-time-to-assigned").cast('float'))
                          .withColumn("Tickets-count",F.col("Tickets-count").cast('int'))
                          .withColumn("Age",F.col("Age").cast('int'))
                          .withColumn("DE-priority",F.col("DE-priority").cast('float'))
                         )

imp = Imputer(inputCols=["Mean-time-to-resolved","Mean-time-to-assigned"], outputCols=["Mean-time-to-resolved","Mean-time-to-assigned"])
df_filtered7 = imp.setStrategy("median").setMissingValue(np.nan).fit(df_filtered7).transform(df_filtered7)


In [0]:
def changeLabels(col,labels= [1,2,3,0],keys=None,mapping={}) :
    if not mapping : 
        if not keys: 
            matching = {i:v for i,v in enumerate(labels)}
        else : 
            matching = {i:v for i,v in zip(keys,labels)}
    else : 
        matching = mapping
    if type (col) == str : 
        col = F.col(col)
    return reduce(lambda y,(src,dest): F.when(col == src,dest ).otherwise(y),matching.items(),None)

In [0]:
#To bin the continous variables MTTR, MTTA and Age since these are having outliers and to drop existing columns
from pyspark.ml.feature import Bucketizer

bins = [-0.1,8,21,40,921]
df_filtered8 = (Bucketizer(splits=bins,inputCol="Mean-time-to-resolved", outputCol='new-MTTR')
                .setHandleInvalid("keep")
                .transform(df_filtered7)
                .withColumn('new-MTTR',changeLabels('new-MTTR',labels=[1,2,3,0]) )
                )
bins = [-0.1,7,16,22,595]
df_filtered9 = (Bucketizer(splits=bins,inputCol="Mean-time-to-assigned", outputCol='new-MTTA')
                .setHandleInvalid("keep")
                .transform(df_filtered8)
                .withColumn('new-MTTA',changeLabels('new-MTTA',labels=[1,2,3,0]) )
                )

bins =  [-0.1,1,8,316]
df_filtered10 = (Bucketizer(splits=bins,inputCol="Tickets-count", outputCol='TC')
                .setHandleInvalid("keep")
                .transform(df_filtered9)
                .withColumn('TC',changeLabels('TC',labels=[1,2,0]) )
                )

bins = [0,10,22,32,922]
df_filtered11 = (Bucketizer(splits=bins,inputCol="Age", outputCol='new-age')
                .setHandleInvalid("keep")
                .transform(df_filtered10)
                .withColumn('new-age',changeLabels('new-age',labels=[1,2,3,0]) )
                )


In [0]:
#to map the levels with the column as per EDA analysis
## This is commented because, when data is filtered for   Status as 'M','R','V' and severity 1,2,3 
## feature column has only one value 'fix' 
## hence, this 'feature' column is removed during pre-processing/cleaning data

#mapping = {'feature':0,'featurette':0,'fix':1,'port':0,'technology':0}
#df_filtered12=df_filtered11.withColumn('Feature',changeLabels('Feature',mapping=mapping) )
df_filtered12 = df_filtered11

In [0]:
#to remove the columns from the dataset as per chi-square contigency test
df_filtered13=df_filtered12.drop('Document','Documents-changed','By-previous-commit-value','Hardware','Data-classification',
                                        'Dev-escape-activity','ORG','Original-activity-when-found','Original-found',
                                        'Is-customer-visible','Test-EDP-activity','WORKAROUND','TPS_flag',
                                        'Project','Breakage','TC','Urgency','Submitter-org','TTA','PSIRT',
                                        'RAT_PSIRT')


In [0]:
print df_filtered13.cache().count(),len(df_filtered13.columns)

14201 29

In [0]:
for c in df_filtered13.columns: 
    print c,
    tmp = df_filtered13.select(F.sum(F.col(c).isNull().cast('int')).alias('a'),F.sum(F.col(c).cast('float').isNull().cast('int')).alias('b')).head()
    print tmp.a,tmp.b,
    if tmp.a < tmp.b : 
        mode = df_filtered13.groupBy(c).count().orderBy(F.desc('count')).head()[c]
        print mode
        df_filtered13 = df_filtered13.fillna(mode,subset=[c])
    print " -"


Activity-when-found 0 14201 Feature/ Functionality Testing
 -
Age 0 0 -
Automated-test 0 14201 N
 -
BadCodeFlag 0 14201 N
 -
Behavior-changed 1 14201 N
 -
Bug-origin 19 14201 Code / Implementation
 -
Component 0 14201 accsw-platform
 -
DE-priority 0 0 -
Found 0 14201 func-test
 -
Identifier 0 14201 CSCvb54995
 -
Impact 0 14201 Features/Functionality
 -
Mean-time-to-assigned 0 0 -
Mean-time-to-resolved 0 0 -
ORG120 0 14201 CSG
 -
Origin 19 14201 base code
 -
Priority 0 12781 NA
 -
Product 0 13750 all
 -
RELNOTE 0 14201 none
 -
Released-code 4 14201 N
 -
RNE-approved 6501 14201 N
 -
Regression 0 14201 N
 -
Severity 0 0 -
Status 0 14201 R
 -
Test-EDP-phase 3121 14201 Feature Test
 -
Tickets-count 0 0 -
org240 0 14201 CSG
 -
new-MTTR 0 0 -
new-MTTA 0 0 -
new-age 0 0 -

In [0]:
#from pyspark.ml import Pipeline
#from pyspark.ml.feature import StringIndexer

#catColumns = ['Bug-origin','Activity-when-found','Automated-test','BadCodeFlag','Behavior-changed','Impact', 
#              'Origin','RELNOTE','Released-code','RNE-approved', 'Status','Test-EDP-phase',  
#              'Regression','Found','org240','Priority','ORG120','Severity', 'Multi-commit-flag', 'Open-status', 'Reti-bug', 'VANALYSIS', 'Related-product']
#indexedColumns = []

#indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df_filtered13) for column in catColumns]


#pipeline = Pipeline(stages=indexers)
#df_r = pipeline.fit(df_filtered13).transform(df_filtered13)

#df_r.show()

In [0]:
#to change from categorical variable to numerical variable using StringIndexer

from pyspark.ml.feature import StringIndexer

catColumns = ['Bug-origin','Activity-when-found','Automated-test','BadCodeFlag','Behavior-changed','Impact','Is-shared-component',
              'Origin','RELNOTE','Released-code','RNE-approved','Sector','Status','Test-EDP-phase','Component','Product',
              'Regression','Found','org240','Priority','ORG120','Severity']
indexedColumns = []
for c in catColumns: 
    try : 
        indexer = StringIndexer(inputCol=c, outputCol=c+"Index") 
        df_filtered13 = indexer.fit(df_filtered13).transform(df_filtered13) 
        indexedColumns.append(c)
    except Exception as e: 
        print c
        print repr(e)
print(indexedColumns)
        

Is-shared-component
IllegalArgumentException()
Sector
IllegalArgumentException()
['Bug-origin', 'Activity-when-found', 'Automated-test', 'BadCodeFlag', 'Behavior-changed', 'Impact', 'Origin', 'RELNOTE', 'Released-code', 'RNE-approved', 'Status', 'Test-EDP-phase', 'Component', 'Product', 'Regression', 'Found', 'org240', 'Priority', 'ORG120', 'Severity']

In [0]:
#to remove the identifer from the dataset inorder to do the feature selection
df_filtered15=df_filtered13
# X = df_filtered5.drop(['Regression'], axis=1).values
# y = df_filtered5['Regression'].values

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator


In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
to_remove = ["Regression", "RegressionIndex","Identifier"]

In [0]:
assembler = VectorAssembler(
    inputCols=[c+"Index" if c in catColumns else c for c in df_filtered15.columns if c not in to_remove],
    outputCol="features")

In [0]:
df_filtered16 = assembler.transform(df_filtered15)

In [0]:
[c+"Index" if c in indexedColumns else c for c in df_filtered15.columns if not c.startswith("RegressionIndex")][40]

Out[ 41 ]: 'Test-EDP-phaseIndex'

In [0]:
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=15).fit(df_filtered16)
df_filtered17 = featureIndexer.transform(df_filtered16)

In [0]:
categoricalFeaturesInfo = { k: max(v.keys()) for k,v in featureIndexer.categoryMaps.items()}

In [0]:

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = df_filtered17.randomSplit([0.7, 0.3])

In [0]:
# Train a RandomForest model.
rf = RandomForestClassifier(featuresCol="indexedFeatures",labelCol='RegressionIndex', maxBins=980)


In [0]:

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, rf])


In [0]:

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "RegressionIndex", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="RegressionIndex", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("accuracy = %g" % (accuracy))



+----------+---------------+--------------------+
prediction|RegressionIndex| features|
+----------+---------------+--------------------+
 0.0| 0.0|(46,[0,1,5,6,7,11...|
 0.0| 0.0|(46,[0,1,6,7,8,9,...|
 0.0| 0.0|(46,[0,1,7,9,10,1...|
 0.0| 0.0|(46,[0,1,6,7,8,11...|
 0.0| 0.0|(46,[0,1,6,7,8,9,...|
+----------+---------------+--------------------+
only showing top 5 rows

accuracy = 0.850722

In [0]:
predictions.groupBy("prediction", "RegressionIndex").count().show()

+----------+---------------+-----+
prediction|RegressionIndex|count|
+----------+---------------+-----+
 1.0| 1.0| 53|
 0.0| 1.0| 612|
 1.0| 0.0| 29|
 0.0| 0.0| 3600|
+----------+---------------+-----+

In [0]:
predictions

Out[ 52 ]: DataFrame[Activity-when-found: string, Age: int, Automated-test: string, BadCodeFlag: string, Behavior-changed: string, Bug-origin: string, Component: string, DE-priority: float, Found: string, Identifier: string, Impact: string, Mean-time-to-assigned: float, Mean-time-to-resolved: float, ORG120: string, Origin: string, Priority: string, Product: string, RELNOTE: string, Released-code: string, RNE-approved: string, Regression: string, Severity: string, Status: string, Test-EDP-phase: string, Tickets-count: int, org240: string, new-MTTR: int, new-MTTA: int, new-age: int, Bug-originIndex: double, Activity-when-foundIndex: double, Automated-testIndex: double, BadCodeFlagIndex: double, Behavior-changedIndex: double, ImpactIndex: double, OriginIndex: double, RELNOTEIndex: double, Released-codeIndex: double, RNE-approvedIndex: double, StatusIndex: double, Test-EDP-phaseIndex: double, ComponentIndex: double, ProductIndex: double, RegressionIndex: double, FoundIndex: double, org240Index: double, PriorityIndex: double, ORG120Index: double, SeverityIndex: double, features: vector, indexedFeatures: vector, rawPrediction: vector, probability: vector, prediction: double]

In [0]:
predictions.show(5)

+-------------------+---+--------------+-----------+----------------+--------------------+---------------+-----------+-----------+----------+--------------------+---------------------+---------------------+-----------+--------------+--------+-------+-------+-------------+------------+----------+--------+------+--------------+-------------+-----------+--------+--------+-------+---------------+------------------------+-------------------+----------------+---------------------+-----------+-----------+------------+------------------+-----------------+-----------+-------------------+--------------+------------+---------------+----------+-----------+-------------+-----------+-------------+--------------------+--------------------+--------------------+--------------------+----------+
Activity-when-found|Age|Automated-test|BadCodeFlag|Behavior-changed| Bug-origin| Component|DE-priority| Found|Identifier| Impact|Mean-time-to-assigned|Mean-time-to-resolved| ORG120| Origin|Priority|Product|RELNOTE|Released-code|RNE-approved|Regression|Severity|Status|Test-EDP-phase|Tickets-count| org240|new-MTTR|new-MTTA|new-age|Bug-originIndex|Activity-when-foundIndex|Automated-testIndex|BadCodeFlagIndex|Behavior-changedIndex|ImpactIndex|OriginIndex|RELNOTEIndex|Released-codeIndex|RNE-approvedIndex|StatusIndex|Test-EDP-phaseIndex|ComponentIndex|ProductIndex|RegressionIndex|FoundIndex|org240Index|PriorityIndex|ORG120Index|SeverityIndex| features| indexedFeatures| rawPrediction| probability|prediction|
+-------------------+---+--------------+-----------+----------------+--------------------+---------------+-----------+-----------+----------+--------------------+---------------------+---------------------+-----------+--------------+--------+-------+-------+-------------+------------+----------+--------+------+--------------+-------------+-----------+--------+--------+-------+---------------+------------------------+-------------------+----------------+---------------------+-----------+-----------+------------+------------------+-----------------+-----------+-------------------+--------------+------------+---------------+----------+-----------+-------------+-----------+-------------+--------------------+--------------------+--------------------+--------------------+----------+
 Acceptance Testing| 3| N| N| N| Port / Sync / Merge| star-sw| 1.0| func-test|CSCvf66870|Features/Function...| 0.0| 2.0|ENT-ROUTING|porting damage| NA| isr| none| N| N| N| 2| R| Feature Test| 0|ENT-ROUTING| 1| 1| 1| 3.0| 12.0| 0.0| 0.0| 0.0| 0.0| 3.0| 0.0| 0.0| 0.0| 0.0| 0.0| 111.0| 10.0| 0.0| 0.0| 1.0| 0.0| 1.0| 1.0|(46,[0,1,5,6,7,11...|(46,[0,1,5,6,7,11...|[15.9767134866262...|[0.79883567433131...| 0.0|
 Acceptance Testing| 10| N| N| N|Code / Implementa...| ie-switch-gps| 3.0| sol-test|CSCvf23664| Reliability| 0.0| 9.0| CSG| base code| NA| moreno| ignore| N| N| N| 3| V| Feature Test| 0| CSG| 2| 1| 2| 0.0| 12.0| 0.0| 0.0| 0.0| 2.0| 0.0| 1.0| 0.0| 0.0| 1.0| 0.0| 538.0| 68.0| 0.0| 5.0| 0.0| 0.0| 0.0| 0.0|(46,[0,1,6,7,8,9,...|(46,[0,1,6,7,8,9,...|[16.4919255101530...|[0.82459627550765...| 0.0|
 Acceptance Testing| 59| N| N| N|Code / Implementa...| accsw-platform| 3.0| func-test|CSCvc82593| Reliability| 3.0| 58.0| CSG| base code| NA|cat4000| none| N| N| N| 2| V| Development| 0| CSG| 0| 1| 0| 0.0| 12.0| 0.0| 0.0| 0.0| 2.0| 0.0| 0.0| 0.0| 0.0| 1.0| 1.0| 0.0| 14.0| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|(46,[0,1,7,9,10,1...|(46,[0,1,7,9,10,1...|[16.9279616291680...|[0.84639808145840...| 0.0|
 Code Analysis| 2| N| N| N|Code / Implementa...|coresw-platform| 3.0|development|CSCvf67776|Features/Function...| 0.0| 1.0| CSG| base code| NA| c9500e| none| N| N| N| 3| R| Feature Test| 0| CSG| 1| 1| 1| 0.0| 9.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 6.0| 53.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0|(46,[0,1,6,7,8,11...|(46,[0,1,6,7,8,11...|[16.5984370040433...|[0.82992185020216...| 0.0|
 Code Analysis| 4| N| N| N|Code / Implementa...| ewlc-client| 3.0|development|CSCvd62591| Maintainability| 0.0| 3.0| WIRELES